# Exemplo de ETL

## Objetivo
- Usando o exemplo simples de ETL, mostrar as funções
  - Criação do Cluster Apache Spark
  - Navegação `Markdown`
  - Sistema de arquivos `workspace`
  - Catalog
  - Histórico de alterações
  - Depuração de variáveis
  - Spark UI

## O que é ETL?

**ETL** é uma sigla que significa **Extract, Transform, Load** (Extração, Transformação e Carga).

![ETL](https://raw.githubusercontent.com/rafael-negrao/laboratorio-databricks/main/imagens/etl.png)

- **Extract (Extração):** Consiste em capturar dados de diversas fontes, como bancos de dados, APIs, arquivos e sistemas legados.

- **Transform (Transformação):** Os dados extraídos são processados e transformados para garantir qualidade, consistência e formato adequado para análise. Isso pode incluir limpeza, agregação, normalização e outras transformações.

- **Load (Carga):** Os dados transformados são carregados em um sistema de destino, como um data warehouse, onde podem ser analisados e utilizados para gerar insights.

## Extract

Criando para repouso dos dados

In [0]:
%sql
create volume if not exists `laboratorio-databricks`;

### Usando pandas

In [0]:
import pandas as pd

pdf_data = pd.read_csv(
    "../dados/owid-covid-data.csv",
    parse_dates=["date"]
)

pdf_data

### Usando shell

In [0]:
%sh
wget -O /Volumes/workspace/default/laboratorio-databricks/covid-data.csv https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv

In [0]:
file = "/Volumes/workspace/default/laboratorio-databricks/covid-data.csv"
df = spark.read.csv(file, header=True, inferSchema=True)

# carregando o dataframe spark a partir de um dataframe pandas
# df = spark.createDataFrame(pdf_data)

Fazendo o print da estrutura de dados do dataframe

In [0]:
df.printSchema()

Exibindo os dados no modo texto, geralmente quando trabalhamos no terminal

In [0]:
df.show(
    5, 
    truncate=False
)

In [0]:
display(df)

Criar o database para recepcionar os dados

In [0]:
%sql
create database if not exists covid_db;

- Exibindo os dados
- Análise exploratória (data profile)

In [0]:
display(df)

Databricks data profile. Run in Databricks to view.

In [0]:
(
  df
    .write
    .mode(saveMode="overwrite")
    .saveAsTable("covid_db.covid_data")
)

## Transform

In [0]:
df_covid = spark.read.table("covid_db.covid_data")

In [0]:
display(
    spark.sql("DESCRIBE DETAIL covid_db.covid_data")
)

In [0]:
display(df_covid)

In [0]:
df_covid.createOrReplaceTempView("covid_data")

In [0]:
%sql
select location, `date`, total_cases, new_cases, total_deaths
  from covid_db.covid_data

In [0]:
df_selected = df.select("location", "date", "total_cases", "new_cases", "total_deaths")

# outra forma de fazer a mesma coisa
df_selected = spark.sql(
"""
select location, `date`, total_cases, new_cases, total_deaths
  from covid_db.covid_data
""")

In [0]:
import pyspark.sql.functions as f

df_selected = df_selected.where(f.col("location") == "United States")
display(df_selected)

In [0]:
df_transformed = (
    df_selected.withColumn(
        "case_fatality_rate",
        f.when(
            (f.col("total_cases").isNotNull()) & (f.col("total_cases") != 0),
            f.col("total_deaths") / f.col("total_cases")
        ).otherwise(f.lit(0.0)) 
    )
)

## Load

In [0]:

(
    df_transformed
    .write
    .mode(saveMode="overwrite")
    .format("delta")
    .saveAsTable("covid_db.covid_transformed")
)

`INFO: Fim do Notebook`